# OIH Queries 

What follows are some example SPARQL queries used in OIH for the test interface


## Setup and inits

### Imports

In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
#import dask, boto3
#import dask.dataframe as dd
import numpy as np
import json
import geopandas
import matplotlib.pyplot as plt
import shapely
# import kglab as kg

oih = "https://ts.collaborium.io/blazegraph/namespace/oih/sparql"
oihdev = "https://ts.collaborium.io/blazegraph/namespace/development/sparql"
oihinvemar = "https://ts.collaborium.io/blazegraph/namespace/invemar/sparql"
oihad = "https://graph.collaborium.io/blazegraph/namespace/aquadocs/sparql"

### Support Functions

In [6]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

## Queries

What follows is a set of queries designed to provide a feel for the OIH graph

## Keywords

In [7]:
rq_keywords = """prefix prov: <http://www.w3.org/ns/prov#>
        PREFIX con: <http://www.ontotext.com/connectors/lucene#>
        PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
        PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX schema: <https://schema.org/>
        PREFIX schemaold: <http://schema.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

       SELECT   ( COUNT(?kw) as ?count) ?kw ?wat ?orgname
        WHERE
        {
           ?s schema:keywords ?kw .
           ?wat rdf:name ?orgname .
           FILTER (?orgname = "Ocean Biodiversity Information System") .
        }
        GROUP BY ?kw ?wat ?orgname
        ORDER BY DESC(?count)
        """


In [8]:
dfkw = get_sparql_dataframe(oihdev, rq_keywords)
dfkw['count'] = dfkw["count"].astype(int) # convert count c to int
dfkw.set_index('kw', inplace=True)
dfkw.head(10)

,count,wat,orgname
kw,,,
Fisheries,14655,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
Biology,6950,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
Occurrence,6819,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
Observation,5311,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
Aquaculture,3932,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
Ecology,3516,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
GCFI,2952,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
Iran,2794,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
Management,2603,https://www.re3data.org/repository/obis,Ocean Biodiversity Information System
